In [ ]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import scipy.sparse as sp
import csv 

#Location of the data_folder 
data_folder      ="/nvme/drive_1/NTDS_Final/"
users            = pd.read_csv(data_folder+"usersdata.csv",delimiter='\t',header=None).values
filtered_users   = pd.read_csv(data_folder+"filtered_users.csv",delimiter=',').values

** I. Split Relations by Type **

In [ ]:
csv_writers = []
for feature in [1,2,3,4,5,6,7]: 
    csv_writers.append(csv.writer(open(data_folder+"filtering/relations_type_"+\
                                       str(feature)+".csv","wt"),delimiter='\t'))

with open(data_folder+"relations.csv", "rt") as f_in:
    reader             = csv.reader(f_in, delimiter="\t")
    
    for i, line in enumerate(reader):    
        node_1             = line[2]  
        node_2             = line[3]

        csv_writers[int(line[4])-1].writerow(line)
         
        if i%1000 is 0:
            print("\r"+str(i) +" relations processed", sep=' ', end='', flush=True)

**II. Radomonly Select Users**

In [ ]:
user_idx_dict          = {}
i                      = 0

for user in users:
    if user[4] is 0 and np.random.rand() < 0.05 :
        user_idx_dict[user[0]]=i
        i+=1
    elif user[4] is 1 and np.random.rand() < 0.05 :
        user_idx_dict[user[0]]=i
        i+=1

** III. Create Adjacency Matrices **

In [ ]:
i = 0 
for feature in [5]:
    with open(data_folder+"filtering/relations_type_"+\
              str(feature)+".csv") as f_in: 
        
        adj     = sp.lil_matrix((len(user_idx_dict),len(user_idx_dict)), dtype='i2')    
        reader  = csv.reader(f_in, delimiter="\t")
        
        for line in reader:    
        
            i+=1  
            
            if int(line[3]) in user_idx_dict and int(line[2]) in user_idx_dict:
                adj[user_idx_dict[int(line[3])],user_idx_dict[int(line[2])]] = 1     
                adj[user_idx_dict[int(line[2])],user_idx_dict[int(line[3])]] = 1
        
            if i%1000 is 0:
                print("\r"+str(i) +" relations processed", sep=' ', end='', flush=True)
        
        csr = adj.tocsr()
        indices_to_keep = np.nonzero(csr.sum(axis=1))[0]
        
        i = 0
        labels = np.zeros((len(indices_to_keep),2))
        for user in users:
            if int(user[0]) in user_idx_dict:
                if user_idx_dict[user[0]] in indices_to_keep: 
                    labels[i] = np.array([i,user[4]])
                    i+=1
                    
        np.save(data_folder+"new_filtering/labels.npy",labels)
        sp.save_npz(data_folder+"new_filtering/adjacency_"+str(feature)+".npz",csr[indices_to_keep,:][:,indices_to_keep])
    

In [ ]:
print("Fraction of spammers in sample : ", labels[:,1].sum()/labels.shape[0])
print("Number of spammers in sample : ",labels[:,1].sum())